# Load Ni-Mo data

In [3]:
from pymatgen.core import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Setup the initial weights for training (If not, the weights for energy and force will be both equal to 1)

In [2]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 100

# Set up the qSNAP and train

In [3]:
from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Mo': {'r': 5.0, 'w': 1}, 'Ni': {'r': 5.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=6, element_profile=element_profile, 
                                   quadratic=True, pot_fit=True)
model = ModelWithSklearn(describer=describer, model=LinearRegression())
qsnap = SNAPotential(model=model)
qsnap.train(train_structures, train_energies, train_forces, sample_weight=weights)

# Predict the energies, forces of training data

In [4]:
df_orig, df_predict = qsnap.evaluate(test_structures=train_structures, 
                                     test_energies=train_energies,
                                     test_forces=train_forces, test_stresses=None)

In [5]:
df_predict[df_predict['dtype'] == 'energy']

,y_orig,n,dtype
0,-1022.381457,144.0,energy
433,-1021.510280,144.0,energy
866,-968.132281,144.0,energy
1299,-974.413236,144.0,energy
1732,-968.107605,144.0,energy
2165,-1009.098989,144.0,energy
2598,-1020.787494,144.0,energy
3031,-958.864667,144.0,energy
3464,-1008.968115,144.0,energy
3897,-972.094278,144.0,energy


# Lattice constant, Elastic constant
### Large error due to limited training data -- 10 structures

In [5]:
from pymatgen.core import Lattice

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [6]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=qsnap)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate([Ni])[0]
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

Ni  C11:  217682520.879883 C12:  7589991.25 C44:  -92536033.9965


In [7]:
Mo_ec_calculator = ElasticConstant(ff_settings=qsnap)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate([Mo])[0]
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

Mo  C11:  -339.481069999747 C12:  -281.969206500333 C44:  287304.423364424


# Load from parameters files

In [9]:
qsnap_loaded = SNAPotential.from_config(param_file='SNAPotential.snapparam', coeff_file='SNAPotential.snapcoeff')

# Energy, force, stress prediction

In [10]:
from maml.apps.pes import EnergyForceStress

efs_calculator = EnergyForceStress(ff_settings=qsnap_loaded)
energy, forces, stresses = efs_calculator.calculate([train_structures[0]])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -1022.38144474744
forces: 
 [[-0.104955    0.602339    0.0869616 ]
 [ 0.566629   -0.416801   -0.767495  ]
 [-0.431159   -0.0925376   0.529552  ]
 [ 0.195108    0.356172   -0.401171  ]
 [-0.609619   -0.0679634  -1.50584   ]
 [ 0.438837    0.0907875  -0.254665  ]
 [-0.697364    0.154224    0.264859  ]
 [-0.141106    0.0327338   0.764932  ]
 [-1.22901    -0.512032    0.242512  ]
 [ 0.684857   -0.33278    -0.464771  ]
 [ 0.0443069  -0.12094     0.0441682 ]
 [-0.802489    0.0899697  -0.591699  ]
 [-0.0910656  -0.132209    0.617263  ]
 [-0.201382   -0.605774   -0.107575  ]
 [ 0.293304   -0.775934   -1.30117   ]
 [-0.628046   -0.540387    0.219001  ]
 [-0.210253   -0.151532   -0.116492  ]
 [-0.223493   -0.0582363  -0.540316  ]
 [-0.0291229  -0.336132    1.15714   ]
 [-0.375756    0.204714   -0.446726  ]
 [-0.715109    0.598777    0.427773  ]
 [-0.224202   -0.00220803  0.0843487 ]
 [-0.240748    1.51498     0.823912  ]
 [ 0.943472   -0.347317   -0.281062  ]
 [-0.0558738  -0.540331    0